In [18]:
import pandas as pd
import numpy as np
import os
import re

# 1. Load Data

In [9]:
folder_path = "../data/"
final_data_path = "../data/annotated_data_final.xlsx"


dfs = []

files = [file for file in os.listdir(folder_path) if file[0].isnumeric()]

for file in files:
    df = pd.read_csv(os.path.join(folder_path, file), index_col = 0)
    dfs.append(df)
else:
    df = pd.read_excel(final_data_path, index_col = 0)
    dfs.append(df)


dfs = pd.concat(dfs, ignore_index=True)


# 중복 제거

In [13]:
dfs.drop_duplicates(inplace = True, ignore_index= True)

In [19]:
english_words_pattern = re.compile(r'\b[a-zA-Z]+\b')
english_mask = dfs['text'].apply(lambda row: bool(english_words_pattern.search(str(row))))
dfs_aft = dfs[~english_mask]

In [22]:
dfs_aft.to_excel("../data/aft_drop.xlsx")

# 데이터 분포 확인

In [24]:
labels = list(range(4))

for label in labels:
    temp_df = dfs_aft[dfs_aft['label'] == label]
    print(f"label {label} has {len(temp_df)} datas")

label 0 has 491 datas
label 1 has 4908 datas
label 2 has 4584 datas
label 3 has 565 datas


In [26]:
# 정상 통화 데이터에서 4000개 추가할 것
 
normal_data = pd.read_excel("../data/normal_data.xlsx", index_col=0)
sampled_data = normal_data.sample(n = 4000, replace = False, random_state = 42, ignore_index = True)

In [39]:
sampled_data.rename(columns = {"SENTENCE" : "text"}, inplace=True)
sampled_data['label'] = np.zeros([len(sampled_data)])
sampled_data['augmented'] = np.zeros([len(sampled_data)])

sampled_data.head(5)

,text,label,augmented
0,내일 와서 계약하겠습니다,0.0,0.0
1,이 제품은 스킨이랑 에센스랑 같이 되죠?,0.0,0.0
2,성인 두 명이에요,0.0,0.0
3,그릴 사용료가 얼마입니까?,0.0,0.0
4,남자 커트 비용은 얼마인가요?,0.0,0.0


In [40]:
total_data = pd.concat([dfs_aft, sampled_data], ignore_index=True)

In [41]:
total_data

,text,label,augmented
0,왜냐하면 지금 이 김지호라는 사람이 전직 은행 사원인데 재직하고 있는 기간 동안에 ...,1.0,1.0
1,왜냐하면 지금사원 이 김지호라는 사람이 전직 은행 직원인데 재직하고 있는 기간 동안...,1.0,1.0
2,왜냐하면 지금 이 김지호라는 사람이 전직 은행 직원인데 재직하고 있는 기간 동안에 ...,1.0,1.0
3,혹시 재정권에서 본인이 개인 경보가 유출이 되었다고 따로 연락받으신 화기 없으시고요,1.0,1.0
4,혹시 금융권화기에서결합 본인이 개인 정보재정가 유출이 되었다고 따로 연락받으신 건 ...,1.0,1.0
...,...,...,...
14543,픽업 오시면 언제쯤 도착하시나요?,0.0,0.0
14544,그냥 솜이에요,0.0,0.0
14545,귤은 한 박스에 얼마에요?,0.0,0.0
14546,조금이라도 드시고 약 드세요,0.0,0.0


In [7]:
total_data = total_data.sample(frac=1).reset_index(drop = True)
total_data.to_excel("final_data.xlsx")